# Maxfilter and time-domain band-pass filtering

(Using the upgraded 2016-versions of `stormdb`-functionality.)

Running `maxfilter` and band-pass filtering (using `mne-python`'s raw-data method `raw.filter`) can be off-loaded to the cluster for load-balanced processing.

## General

### On the naming logic for the study

There are only two blocks (`equi` and `opt`). Data will never be compared at the sensor level between the two blocks, so each can be head motion compensated to its initial head position (basic `maxfilter` usage).

### Use a development version of `mne-python` and `stormdb-python`

__NB:__ The following only needs to be run once, on the command line of any interactive server.

This way the version of the analysis code can be kept with the scripts using them, for future reproduction of results. To get the code, and to make a "snapshot" of the state of the code (in case you modify anything accidentally), open a terminal in a remote desktop and (suggestion):

```bash
cd /projects/MINDLAB2016_MEG-Language-PD-DBS/misc
git clone https://github.com/mne-tools/mne-python.git
cd mne-python
git checkout -b snapshot_20160803

cd /projects/MINDLAB2016_MEG-Language-PD-DBS/misc
git clone https://github.com/meeg-cfin/stormdb-python.git
cd stormdb-python
git fetch origing refactor_submit:refactor_submit
git checkout -b refactor_submit
```

### Output file and folder names

Remember to use the project `scratch` folder for output, and make it easy to clean up!

In [1]:
from os.path import join, basename
proj_name = 'MINDLAB2016_MEG-Language-PD-DBS'
scratch_folder = join('/projects', proj_name, 'scratch')
mf_folder = join(scratch_folder, 'maxfilter')  # for maxfilter output
scripts_folder = join('/projects', proj_name, 'scripts')
misc_folder = join('/projects', proj_name, 'misc')
trans_folder = join(scratch_folder, 'trans')  # for transforms

In [2]:
# Modify path by adding the mne-python folder from scripts to the beginning.
import sys
sys.path = [join(misc_folder, 'mne-python'),
            join(misc_folder, 'stormdb-python')] + sys.path

### Load libraries

In Python, we have to load what we need!

In [3]:
from stormdb.access import Query
from stormdb.process import Maxfilter
from mne.io import Raw
from mne.bem import fit_sphere_to_headshape
import warnings
import os
import re
import numpy as np

In [4]:
# silence mne a bit
from mne.utils import set_log_level
set_log_level('ERROR')

### Constant parameters

Place parameters here you might want to play with, such as tSSS buffer length and correlation limit. Output folders will be automatically generated to reflect these.

In [5]:
tsss_buffer_len = 16
tsss_corr_lim = 0.96
# if you know that some channels are bad or flat, enter them here
# in the form ['2511', '2241']
static_bad_chans = ['2511'] 

### Finding the data

Instead of accessing raw files directly, use the database query functions to get to files.

Note that `get_subjects` by default only returns a list of non-excluded subjects, this may be useful.

In [6]:
qr = Query(proj_name)
subs = qr.get_subjects()

These are the subjects in the database

In [7]:
for ii, ss in enumerate(subs):
    print('{0}: {1}'.format(ii, ss))

0: 0001_JNC
1: 0002_FYX


In [8]:
len(subs)

2

## Select subject

Instead of writing a script that loops over subjects (could be done), this notebook requires you to select one subject at a time. The advantage is that you then sanity-check each stage before continuing. The final `submit_to_cluster` commands do not block the notebook: you can immediately go forward with the next subject. New submissions will simply be added to the queue.

To make this semi-automatic, the variable `cur_sub_index` will be incremented by 1 every time the `submit`-command is issued at the end of this notebook. You can also just manually set the index to what you want.

In [9]:
cur_sub_index = 1  # see below for the meaning of this
cur_sub = subs[cur_sub_index]
print('Current subject: {sub:s}'.format(sub=cur_sub))

Current subject: 0002_FYX


In [10]:
sub_specific_bad_chans = []  # empty list if no (more) bad chans

In [11]:
# Alternatively, just state the ID of the subject
# cur_sub = '0008'

## Calculating the head positions

Here we want to calculate the average initial head position and use movecomp to correct head motion to that origin.

In [12]:
description = '*equi|*optim*'
DATAblocks = qr.filter_series(description=description, subjects=cur_sub,
                              modalities='MEG')

if len(DATAblocks) != 2:
    raise RuntimeError('Not all 2 blocks found for {0}, please check!'.\
                       format(cur_sub))
for ib in range(len(DATAblocks)):
    print('{:d}: {:s}'.format(ib + 1, DATAblocks[ib]['path']))

1: /projects/MINDLAB2016_MEG-Language-PD-DBS/raw/0002/20160802_000000/MEG/001.optimum/files
2: /projects/MINDLAB2016_MEG-Language-PD-DBS/raw/0002/20160802_000000/MEG/002.equi/files


## Fit head origin for SSS expansion

Any info will do, since the digitization points are the same for all blocks. We'll take the first one. NB: Only use EEG locations, since head points only on face!

In [13]:
info = Raw(os.path.join(DATAblocks[0]['path'], DATAblocks[0]['files'][0]),
           preload=False).info
set_log_level('INFO')
rad, origin_head, ori_dev = fit_sphere_to_headshape(info,
                                                    dig_kinds='eeg',
                                                    units='mm')
set_log_level('ERROR')

Fitted sphere radius:         87.9 mm
Origin head coordinates:      2.4 2.6 42.9 mm
Origin device coordinates:    0.2 9.4 -7.8 mm


## Initiate Maxfilter-object

In [14]:
mf = Maxfilter(proj_name, bad=static_bad_chans + sub_specific_bad_chans)

### Build maxfilter commands for all the blocks

First set some of the options (leave others as default).

In [15]:
mfopts = dict(
    origin = '{:.1f} {:.1f} {:.1f}'.format(*tuple(origin_head)),  # mm
    frame = 'head',
    force = True,  # overwrite if needed
    autobad = 'on',  # or use xscan first
    st = True,  # use tSSS
    st_buflen = tsss_buffer_len,  # parameter set in beg. of notebook
    st_corr = tsss_corr_lim,  # parameter set in beg. of notebook
    movecomp = True,
    trans = None,  # compensate to mean initial head position (saved to file),
                              # or use None for initial head position
    logfile = None,  # we replace this in each loop
    hp = None,  # head positions, replace in each loop
    n_threads = 4  # number of parallel threads to run on
    )

mne-python likes raw and raw-like (tsss) files that are part of a long (>2GB) continuous acquisition to follow the convention:

1. `filename_raw_tsss.fif` (first file)
1. `filename_raw_tsss-1.fif` (second file, etc.)


In [16]:
out_folder = join(mf_folder,
                  'tsss_st{:d}_corr{:.0f}'.\
                      format(mfopts['st_buflen'],
                             np.round(100 * mfopts['st_corr'])),
                  cur_sub)

# Ensure that output path exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
print('Output folder: {:s}'.format(out_folder))

Output folder: /projects/MINDLAB2016_MEG-Language-PD-DBS/scratch/maxfilter/tsss_st16_corr96/0002_FYX


Finally, for each block, loop over the associated files (a single file if the data size is less than 2 GB), and:
* define input file name
* determine the name of the block ("series")
* define the output file path by combining the output path with a name based on the block name, suffixed by "`_raw_tsss.fif`"
    * if multiple-file acquisition, add the Elekta-standard suffixes
* define the output log- and head position-file names
* build the `maxfilter`-command based on all the definitions above

In [17]:
for blockno, bl in enumerate(DATAblocks):
    for fileno, fil in enumerate(bl['files']):
        in_fname = join(bl['path'], bl['files'][fileno])
        
        series_name = re.search('(equi|optimum)',
                                bl['seriename']).group(1)
        
        out_fname = join(out_folder, '{:s}_raw_tsss.fif'.format(series_name))
        if fileno > 0:  # data size > 2 GB
            out_fname = out_fname[:-4] + '-{:d}.fif'.format(fileno)
           
        mfopts['logfile'] = out_fname[:-3] + 'log'
        mfopts['hp'] = out_fname[:-3] + 'pos'
        
        mf.build_cmd(in_fname, out_fname, **mfopts)

## Submit to isis for processing

We need to use the `isis.q`, since at present that is the only queue where `maxfilter` is available.

First check that you think sane things will happen, if you like.

To comment out a line in python, prefix the line with a '# '

In [18]:
# This is not executed, but the line below is
mf.check_input_output_mapping()
mf.print_input_output_mapping()

All inputs readable & outputs writable.
/projects/MINDLAB2016_MEG-Language-PD-DBS/raw/0002/20160802_000000/MEG/001.optimum/files/PROJ0252_SUBJ0002_SER001_FILESNO001.fif
	--> /projects/MINDLAB2016_MEG-Language-PD-DBS/scratch/maxfilter/tsss_st16_corr96/0002_FYX/optimum_raw_tsss.fif
/projects/MINDLAB2016_MEG-Language-PD-DBS/raw/0002/20160802_000000/MEG/002.equi/files/PROJ0252_SUBJ0002_SER002_FILESNO001.fif
	--> /projects/MINDLAB2016_MEG-Language-PD-DBS/scratch/maxfilter/tsss_st16_corr96/0002_FYX/equi_raw_tsss.fif


If in doubt, uncomment this line to see the actual commands that will execute:

In [19]:
# mf.commands

### Ready to rock

In [45]:
mf.submit()

Cluster job submitted, job ID: 3550337
Cluster job submitted, job ID: 3550338


### Checking what's going on

In a terminal:

```bash
qstat
```

shows all _your_ running jobs. For _every/anyone's_ jobs, run

```bash
qstat -u "*"
```

You can also just "ask" the `mf`-object what its status is:

In [50]:
mf.status

#1 (3550337): Running on isis (isis.q)
#2 (3550338): Running on isis (isis.q)


To kill a submitted (or even running) job

In [22]:
# mf.kill(3550319)

To kill all submitted jobs:

In [23]:
# mf.kill()

To kill a job in the shell:

```
qdel JOB_NUMBER
```

or for all jobs (in your name):

```
qdel *
```

## Band-pass filter the maxfiltered data

Use the information in the input-output mapping of the previous analysis stage to feed into the next.

In [20]:
mf.info['io_mapping']

[{'input': u'/projects/MINDLAB2016_MEG-Language-PD-DBS/raw/0002/20160802_000000/MEG/001.optimum/files/PROJ0252_SUBJ0002_SER001_FILESNO001.fif',
  'output': u'/projects/MINDLAB2016_MEG-Language-PD-DBS/scratch/maxfilter/tsss_st16_corr96/0002_FYX/optimum_raw_tsss.fif'},
 {'input': u'/projects/MINDLAB2016_MEG-Language-PD-DBS/raw/0002/20160802_000000/MEG/002.equi/files/PROJ0252_SUBJ0002_SER002_FILESNO001.fif',
  'output': u'/projects/MINDLAB2016_MEG-Language-PD-DBS/scratch/maxfilter/tsss_st16_corr96/0002_FYX/equi_raw_tsss.fif'}]

In [21]:
from stormdb.process import MNEPython

In [22]:
mnep = MNEPython(proj_name)

In [23]:
l_freq = 1.
h_freq = 40.
filter_folder = join(scratch_folder, 'filtered')

In [24]:
out_folder = join(filter_folder,
                  '{0:.0f}-{1:.0f}'.format(l_freq or 'None',
                                   h_freq or None), cur_sub)

# Ensure that output path exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
print('Output folder: {:s}'.format(out_folder))

Output folder: /projects/MINDLAB2016_MEG-Language-PD-DBS/scratch/filtered/1-40/0002_FYX


In [25]:
for curmap in mf.info['io_mapping']:
    in_fname = curmap['output']
    out_fname = join(out_folder, basename(curmap['output'])[:-4] + '_filt_raw.fif')
    mnep.raw_filter(in_fname, out_fname, l_freq, h_freq)

In [30]:
mnep.submit()

Cluster job submitted, job ID: 3550332
Cluster job submitted, job ID: 3550333


In [44]:
mnep.status

#1 (3550332): Job completed
#2 (3550333): Job completed
